# Checking RT-Scheduled Trips Table 

In [1]:
import utils
import shared_utils
from dla_utils import _dla_utils as dla_utils
from shared_utils import styleguide, altair_utils, portfolio_utils

from siuba import *
import pandas as pd

from IPython.display import display, Markdown, HTML

import altair as alt

import os
os.environ["CALITP_BQ_MAX_BYTES"] = str(800_000_000_000) ## 800GB?

from calitp.tables import tbl

from calitp import query_sql, magics


/opt/conda/lib/python3.10/site-packages/geopandas/_compat.py:112: UserWarning: The Shapely GEOS version (3.10.3-CAPI-1.16.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [9]:
df = query_sql(f'''
SELECT * FROM `cal-itp-data-infra-staging.natalie_views.gtfs_rt_vs_schedule_trips_transitcenter_sample`
''')

In [10]:
df['service_date'] = pd.to_datetime(df['service_date'])
df['weekday'] = pd.Series(df.service_date).dt.day_name()    
df['month'] =  pd.Series(df.service_date).dt.month_name()

In [11]:
df.head()

,calitp_itp_id,agency_name,calitp_url_number,route_id,route_short_name,service_date,calitp_extracted_at,calitp_deleted_at,num_sched,num_vp,pct_w_vp,weekday,month
0,282,San Francisco Municipal Transportation Agency,1,M,M,2022-09-17,2022-02-16,2099-01-01,164,0,0.00,Saturday,September
1,282,San Francisco Municipal Transportation Agency,1,M,M,2022-09-18,2022-02-16,2099-01-01,164,0,0.00,Sunday,September
2,4,AC TRANSIT,0,U,U,2022-08-11,2022-03-02,2099-01-01,11,0,0.00,Thursday,August
3,4,AC TRANSIT,0,U,U,2022-09-06,2022-03-02,2099-01-01,11,0,0.00,Tuesday,September
4,4,AC TRANSIT,1,U,U,2022-08-15,2022-03-01,2099-01-01,11,0,0.00,Monday,August


## Check URLs

In [12]:
(utils.get_agg_pct(df,
                groupings= ['calitp_itp_id','agency_name', 'calitp_url_number'],
                sum_sched='num_sched',
                sum_vp='num_vp',
               ))>>arrange(_.agency_name, _.calitp_url_number)

,calitp_itp_id,agency_name,calitp_url_number,num_sched,num_vp,avg
1,4,AC TRANSIT,0,313342,256452,0.82
2,4,AC TRANSIT,1,313342,256436,0.82
3,278,MTS,0,226709,199510,0.88
4,278,San Diego International Airport,0,2040,0,0.00
5,282,San Francisco Municipal Transportation Agency,0,538884,0,0.00
0,282,San Francisco Municipal Transportation Agency,1,538884,429604,0.80


To drop:
* AC Transit url:1?
* San Diego International Airport url: 0
* SFMTA url: 0

In [14]:
table = (utils.get_correct_url(df))

In [21]:
(utils.get_agg_pct(table,
                groupings= ['calitp_itp_id','agency_name', 'calitp_url_number'],
                sum_sched='num_sched',
                sum_vp='num_vp',
               ))>>arrange(_.agency_name, _.calitp_url_number)

,calitp_itp_id,agency_name,calitp_url_number,num_sched,num_vp,avg
1,4,AC TRANSIT,1,313342,256436,0.82
2,278,MTS,0,226709,199510,0.88
0,282,San Francisco Municipal Transportation Agency,1,538884,429604,0.80


In [22]:
(utils.get_agg_pct(table,
                groupings= ['calitp_itp_id','agency_name', 'month'],
                sum_sched='num_sched',
                sum_vp='num_vp',
               ))>>arrange(_.agency_name, _.month)

,calitp_itp_id,agency_name,month,num_sched,num_vp,avg
1,4,AC TRANSIT,August,159194,129725,0.81
2,4,AC TRANSIT,September,154148,126711,0.82
5,278,MTS,August,206889,182020,0.88
4,278,MTS,September,19820,17490,0.88
3,282,San Francisco Municipal Transportation Agency,August,274758,217202,0.79
0,282,San Francisco Municipal Transportation Agency,September,264126,212402,0.80


## Add Alerts

In [26]:
def get_alert_data():
    alerts = query_sql(
        f"""
        SELECT * FROM `cal-itp-data-infra.staging.stg_rt__service_alerts`
        WHERE date BETWEEN '2022-08-01' AND '2022-09-30'
        AND effect = 'NO_SERVICE'
        AND (calitp_itp_id in (4, 278, 282))
                """)
    return alerts
                

In [30]:
# alerts = get_alert_data()

In [31]:
## hit query limit
# alerts

In [29]:
table

,calitp_itp_id,agency_name,calitp_url_number,route_id,route_short_name,service_date,calitp_extracted_at,calitp_deleted_at,num_sched,num_vp,pct_w_vp,weekday,month
0,282,San Francisco Municipal Transportation Agency,1,M,M,2022-09-17,2022-02-16,2099-01-01,164,0,0.00,Saturday,September
1,282,San Francisco Municipal Transportation Agency,1,M,M,2022-09-18,2022-02-16,2099-01-01,164,0,0.00,Sunday,September
4,4,AC TRANSIT,1,U,U,2022-08-15,2022-03-01,2099-01-01,11,0,0.00,Monday,August
5,4,AC TRANSIT,1,U,U,2022-09-02,2022-03-01,2099-01-01,11,0,0.00,Friday,September
8,4,AC TRANSIT,1,U,U,2022-08-18,2022-03-01,2099-01-01,11,0,0.00,Thursday,August
...,...,...,...,...,...,...,...,...,...,...,...,...,...
23817,282,San Francisco Municipal Transportation Agency,1,38,38,2022-09-01,2022-02-16,2099-01-01,257,254,0.99,Thursday,September
23818,282,San Francisco Municipal Transportation Agency,1,14R,14R,2022-09-14,2022-02-16,2099-01-01,262,254,0.97,Wednesday,September
23819,282,San Francisco Municipal Transportation Agency,1,8,8,2022-09-05,2022-02-16,2099-01-01,262,255,0.97,Monday,September
23820,282,San Francisco Municipal Transportation Agency,1,38,38,2022-09-07,2022-02-16,2099-01-01,257,255,0.99,Wednesday,September


## Add Trip Updates

In [37]:
analysis_date1 = '2022-09-01'
analysis_date2 = '2022-09-30'
# itp_ids = [4, 278, 282]


In [60]:
## no more query
# updates_282 = query_sql(f'''
# SELECT *
# FROM `cal-itp-data-infra-staging.natalie_staging.stg_rt__trip_updates`
# WHERE date BETWEEN '{analysis_date1}' AND '{analysis_date2}'
# AND calitp_itp_id IN (282) AND calitp_url_number = 1
# AND trip_schedule_relationship != 'SCHEDULED'
# LIMIT 1000
# ''')

In [59]:
# updates_282

In [41]:


updates_4 = query_sql(f'''
SELECT *
FROM `cal-itp-data-infra-staging.natalie_staging.stg_rt__trip_updates`
WHERE date BETWEEN '{analysis_date1}' AND '{analysis_date2}'
AND calitp_itp_id IN (4)
LIMIT 2000
''')

In [63]:
updates_4>>group_by(_.date, _.trip_route_id, _.trip_id) >>count(_.trip_schedule_relationship)>>arrange(-_.trip_route_id)

,date,trip_route_id,trip_id,trip_schedule_relationship,n
549,2022-09-01,O,10784020,SCHEDULED,3
550,2022-09-01,O,10869020,SCHEDULED,4
551,2022-09-01,O,12051020,SCHEDULED,4
552,2022-09-01,O,12541020,SCHEDULED,4
553,2022-09-01,O,2099020,SCHEDULED,4
...,...,...,...,...,...
5,2022-09-01,10,234020,SCHEDULED,3
6,2022-09-01,10,6662020,SCHEDULED,4
7,2022-09-01,10,7909020,SCHEDULED,3
8,2022-09-01,10,9445020,SCHEDULED,4


In [67]:
(updates_4
 >>filter(_.trip_schedule_relationship != 'SCHEDULED')
 >>group_by(_.date, _.trip_route_id, _.trip_schedule_relationship)
 >>summarize(n = _.trip_id.nunique())
 >>arrange(-_.trip_route_id))


,date,trip_route_id,trip_schedule_relationship,n
13,2022-09-01,NL,CANCELED,2
12,2022-09-01,97,CANCELED,2
11,2022-09-01,88,CANCELED,2
10,2022-09-01,72R,CANCELED,6
9,2022-09-01,72,CANCELED,1
8,2022-09-01,7,ADDED,1
6,2022-09-01,57,ADDED,1
7,2022-09-01,57,CANCELED,3
5,2022-09-01,51B,CANCELED,1
4,2022-09-01,40,CANCELED,2


* final table might look like the above cell

In [52]:
updates_4.trip_schedule_relationship.value_counts()

SCHEDULED    1896
CANCELED       96
ADDED           8
Name: trip_schedule_relationship, dtype: int64

In [64]:
updates_4>>filter(_.trip_schedule_relationship=='ADDED')>>count(_.trip_id)

,trip_id,n
0,-1609608939,4
1,-2054354814,4


In [68]:
## SQL query to agg trip updates
# query_sql(f'''
# SELECT 
#   calitp_itp_id,
#   calitp_url_number,
#   date,
#   trip_route_id,
#   trip_schedule_relationship,
# COUNT(DISTINCT trip_id) AS Count,
# FROM `cal-itp-data-infra.staging.stg_rt__trip_updates` 
# WHERE calitp_itp_id = 4 AND calitp_url_number = 1 AND
#   date BETWEEN '2022-09-01' AND '2022-09-30' AND
#   trip_schedule_relationship <> 'SCHEDULED'
# GROUP BY calitp_itp_id, calitp_url_number, date, trip_route_id, trip_schedule_relationship
# ''')